In [163]:
import numpy as np
from numpy.polynomial import polynomial as P

def innerProductPolynoms(f1, f2, xmin, xmax):
        f1mf2 = P.polysub(f1,f2)
        f1mf2 = P.polymul(f1mf2,f1mf2)
        val = P.polyint(f1mf2,lbnd=xmin)
        #print("val", val)
        ip = np.abs(val[0] + val[1]*(xmax-xmin)+val[2]*(xmax-xmin)**2+val[3]*(xmax-xmin)**3+val[4]*(xmax-xmin)**4+val[5]*(xmax-xmin)**5)
        return np.sqrt(ip)/(xmax-xmin)
    
def associatePolynomAndPrior(res, priors):
    ass_list = []
    polynoms = res[-1]
    
    for iprior,prior in enumerate(priors):
        for ipol,polynom in enumerate(polynoms):
            a0_err = abs(polynom["f"].c[2]-prior[0])
            a1_err = abs(polynom["f"].c[1]-prior[1])
            a2_err = abs(polynom["f"].c[0]-prior[2])
            #print(polynom["f"].c,prior, a0_err, a1_err, a2_err)
            if a0_err < 8 and a1_err < 1 and a2_err < 0.1:
                ass_list.append((iprior,ipol))
                
    return ass_list

def ComputeCoeffErrors(res, prior, ipolynom):
    #print(res[-1][ipolynom]["f"],prior)
    a0_err = np.zeros([len(res), 3])
    a1_err = np.zeros([len(res), 3])
    a2_err = np.zeros([len(res), 3])
    for iframe,frame in enumerate(res):
        a0_err[iframe,0] = abs(frame[ipolynom]["f"].c[2] - prior[0])
        a1_err[iframe,1] = abs(frame[ipolynom]["f"].c[1] - prior[1])
        a2_err[iframe,2] = abs(frame[ipolynom]["f"].c[0] - prior[2])
        
    a0_mean_err = np.mean(a0_err)
    a1_mean_err = np.mean(a1_err)
    a2_mean_err = np.mean(a2_err)
    print(f"ComputeCoeffErrors: mean err for prior {prior} = ({a0_mean_err},{a1_mean_err},{a2_mean_err})")
    
    
def ComputePolynomDistance(res, prior, ipolynom):
    dist = np.zeros(len(res))
    for iframe,frame in enumerate(res):
        
        f1 = (prior[0],prior[1],prior[2])
        f2 = (frame[ipolynom]["f"].c[2],frame[ipolynom]["f"].c[1],frame[ipolynom]["f"].c[0])
        xmin = max(prior[3],frame[ipolynom]["x_start"])
        xmax = min(prior[4],frame[ipolynom]["x_end"])
        dist[iframe] = innerProductPolynoms(f1,f2,xmin,xmax)
        #print("f1",f1,"f2",f2,"xmin",xmin,"xmax",xmax,"dist[iframe]",dist[iframe])
        
    dist_mean_err = np.mean(dist)
    dist_max_err = np.max(dist)
    print(f"ComputePolynomDistance: dist_mean_err for prior {prior} = {dist_mean_err}")
    print(f"ComputePolynomDistance: dist_max_err for prior {prior} = {dist_max_err}")
    
def ComputeOverlappingArea(res, prior, ipolynom):
    overlap = np.zeros(len(res))
    for iframe,frame in enumerate(res):
        xmin = max(prior[3],frame[ipolynom]["x_start"])
        xmax = min(prior[4],frame[ipolynom]["x_end"])
        
        xminmin = min(prior[3],frame[ipolynom]["x_start"])
        xmaxmax = max(prior[4],frame[ipolynom]["x_end"])
        overlap[iframe] = np.abs(xmax-xmin)/np.abs(xmaxmax-xminmin)
        
    overlap_mean = np.mean(overlap)
    overlap_min = np.min(overlap)
    print(f"ComputeOverlappingArea: overlap_mean for prior {prior} = {overlap_mean}")
    print(f"ComputeOverlappingArea: overlap_min for prior {prior} = {overlap_min}")
    
def ComputeNonOverlappingArea(res, prior, ipolynom):
    nonoverlap = np.zeros(len(res))
    for iframe,frame in enumerate(res):
        dmin = prior[3]-frame[ipolynom]["x_start"]
        dmax = frame[ipolynom]["x_end"] - prior[4]
        
        dmin = dmin if dmin > 0 else 0
        dmax = dmin if dmax > 0 else 0
        
        nonoverlap[iframe] = dmax + dmin
        
    nonoverlap_mean = np.mean(nonoverlap)
    nonoverlap_max = np.max(nonoverlap)
    print(f"ComputeNonOverlappingArea: onon_verlap_mean for prior {prior} = {nonoverlap_mean}")
    print(f"ComputeNonOverlappingArea: non_overlap_max for prior {prior} = {nonoverlap_max}")
        

def analyzeResults(res, priors):
    ass_list = associatePolynomAndPrior(res, priors)
    for iprior,prior in enumerate(priors):
        ComputeCoeffErrors(res, priors[ass_list[iprior][0]], ass_list[iprior][1])
        ComputePolynomDistance(res, priors[ass_list[iprior][0]], ass_list[iprior][1])
        ComputeOverlappingArea(res, priors[ass_list[iprior][0]], ass_list[iprior][1])
        ComputeNonOverlappingArea(res, priors[ass_list[iprior][0]], ass_list[iprior][1])

In [ ]:
%load_ext autoreload
%autoreload 2
from simulation import DynamicSimulation
from tracker import StaticTracker
model = StaticTracker()
prior = [(0, 0, 0.00001, 3, 40, True), 
        (10, 0.009, -0.004, 50, 90, True),
        (40, 0, -0.01, 1, 40, False)]
#prior = [(0, 0, 0.0001, 3, 40)]
sim = DynamicSimulation(model,prior=prior)
res = sim.run(N=20)

analyzeResults(res,prior)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
frame 0
data_size 120 zw.shape (144, 2)
prior [{'c': (0, 0, 1e-05), 'xmin': 3, 'xmax': 40, 'fx': True}, {'c': (10, 0.009, -0.004), 'xmin': 50, 'xmax': 90, 'fx': True}, {'c': (40, 0, -0.01), 'xmin': 1, 'xmax': 40, 'fx': False}]
Number of point tracks before run() 0
Number of extended tracks before run() 0
len(unique_polynoms) 0
frame 1
data_size 120 zw.shape (144, 2)
prior [{'c': (0, 0, 1e-05), 'xmin': 3, 'xmax': 40, 'fx': True}, {'c': (10, 0.009, -0.004), 'xmin': 50, 'xmax': 90, 'fx': True}, {'c': (40, 0, -0.01), 'xmin': 1, 'xmax': 40, 'fx': False}]
Number of point tracks before run() 144
Number of extended tracks before run() 0


/home/kfir/workspace/RadarLocalization/tracker.py:86: RuntimeWarning: divide by zero encountered in double_scalars
  


len(unique_polynoms) 0
frame 2
data_size 120 zw.shape (144, 2)
prior [{'c': (0, 0, 1e-05), 'xmin': 3, 'xmax': 40, 'fx': True}, {'c': (10, 0.009, -0.004), 'xmin': 50, 'xmax': 90, 'fx': True}, {'c': (40, 0, -0.01), 'xmin': 1, 'xmax': 40, 'fx': False}]
Number of point tracks before run() 217
Number of extended tracks before run() 0
Opening flipped polynom!!!
created an extended object! [ 3.99329261e+01 -1.62731271e-02 -9.54765142e-03 -1.05190393e-01
  3.88773435e+01]
created an extended object! [ 1.32904146e-01 -9.94627579e-03  3.89685568e-04  3.22285968e+00
  3.89690509e+01]
created an extended object! [ 1.10465027e+01 -5.09971332e-02 -3.56436712e-03  5.01147839e+01
  8.99877007e+01]
Tracks are similar! do not open a new trk (0.5188736024160776, -0.006033381062303229, 0.0003896855683518883) (0.0, 0.005936866031576634, -0.0001941517599232297)
Tracks are similar! do not open a new trk (-1.3518073515243636, -0.4111250524215142, -0.003564367124252374) (0.0, -0.41086768756089875, -0.003538837

/home/kfir/workspace/RadarLocalization/tracker.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/home/kfir/workspace/RadarLocalization/tracker.py:86: RuntimeWarning: divide by zero encountered in double_scalars
  


Updating extended object track =  2
Updating extended object track =  0
Updating extended object track =  0
Updating extended object track =  2
Updating extended object track =  2
Updating extended object track =  2
Updating extended object track =  2
Updating extended object track =  1
Updating extended object track =  2
Updating extended object track =  0
Opening flipped polynom!!!
Tracks are similar! do not open a new trk (-0.044846978335037264, -0.02078838776800429, -0.00990584608203207) (0.0, -0.003133472805165932, -0.010340015592612445)
Tracks are similar! do not open a new trk (-0.05353620581535944, -0.005870583788527432, 0.00036739175677183786) (0.0, -0.031411667001039666, 0.0007790311684058984)
Tracks are similar! do not open a new trk (-1.0086924665523176, -0.42422425906864686, -0.0036277580702726085) (0.0, -0.3895114293168348, -0.004077894338741658)
len(unique_polynoms) 3
frame 4
data_size 120 zw.shape (144, 2)
prior [{'c': (0, 0, 1e-05), 'xmin': 3, 'xmax': 40, 'fx': True}, 

...

/home/kfir/workspace/RadarLocalization/tracker.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/home/kfir/workspace/RadarLocalization/tracker.py:86: RuntimeWarning: divide by zero encountered in double_scalars
  


Updating extended object track =  2
Updating extended object track =  2
Opening flipped polynom!!!
Tracks are similar! do not open a new trk (-0.15785057021487137, -0.0446649424139416, -0.009853319717488934) (0.0, -0.03559715187515644, -0.010148500172141037)
Tracks are similar! do not open a new trk (0.024876785984565355, 0.0015644989922342567, 0.0002263981359569325) (0.0, 0.01722582470518356, -0.0003940334482552435)
Tracks are similar! do not open a new trk (-0.838199206978067, -0.4310484381518456, -0.0036434436612869238) (0.0, -0.40437327706485954, -0.00404213535812958)
len(unique_polynoms) 1
frame 7
data_size 120 zw.shape (144, 2)
prior [{'c': (0, 0, 1e-05), 'xmin': 3, 'xmax': 40, 'fx': True}, {'c': (10, 0.009, -0.004), 'xmin': 50, 'xmax': 90, 'fx': True}, {'c': (40, 0, -0.01), 'xmin': 1, 'xmax': 40, 'fx': False}]
Number of point tracks before run() 204
Number of extended tracks before run() 3


/home/kfir/workspace/RadarLocalization/tracker.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/home/kfir/workspace/RadarLocalization/tracker.py:86: RuntimeWarning: divide by zero encountered in double_scalars
  


Updating extended object track =  0
Updating extended object track =  2
Updating extended object track =  2
Opening flipped polynom!!!
Tracks are similar! do not open a new trk (0.2231821470596061, 0.01623330594889194, -0.010142181655411395) (0.0, 0.05973530228010515, -0.012184706411099764)
Tracks are similar! do not open a new trk (-0.1094538313169183, 0.001717642347343536, 0.00022639813595693234) (0.0, -0.029981800949444377, 0.0013138625011337873)
Tracks are similar! do not open a new trk (-1.0912410299773525, -0.37916816835549916, -0.004726483892176425) (0.0, -0.35707886389932403, -0.005042917434939899)
len(unique_polynoms) 2
frame 8
data_size 120 zw.shape (144, 2)
prior [{'c': (0, 0, 1e-05), 'xmin': 3, 'xmax': 40, 'fx': True}, {'c': (10, 0.009, -0.004), 'xmin': 50, 'xmax': 90, 'fx': True}, {'c': (40, 0, -0.01), 'xmin': 1, 'xmax': 40, 'fx': False}]
Number of point tracks before run() 249
Number of extended tracks before run() 3


/home/kfir/workspace/RadarLocalization/tracker.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [151]:
analyzeResults(res,prior)

ComputeCoeffErrors: mean err for prior (0, 0, 1e-05, 3, 40, True) = (0.0540064797381548,0.0038093326758344252,0.00017653161518752)
ComputePolynomDistance: dist_mean_err for prior (0, 0, 1e-05, 3, 40, True) = 0.017642605034590263
ComputePolynomDistance: dist_max_err for prior (0, 0, 1e-05, 3, 40, True) = 0.03310096710012558
ComputeOverlappingArea: overlap_mean for prior (0, 0, 1e-05, 3, 40, True) = 0.8799734901110551
ComputeOverlappingArea: overlap_min for prior (0, 0, 1e-05, 3, 40, True) = 0.8238002411741217
ComputeNonOverlappingArea: onon_verlap_mean for prior (0, 0, 1e-05, 3, 40, True) = 0.0
ComputeNonOverlappingArea: non_overlap_max for prior (0, 0, 1e-05, 3, 40, True) = 0.0
ComputeCoeffErrors: mean err for prior (10, 0.009, -0.004, 50, 90, True) = (2.547341653281811,0.07734229053018243,0.0005423252901726406)
ComputePolynomDistance: dist_mean_err for prior (10, 0.009, -0.004, 50, 90, True) = 0.07829325941546049
ComputePolynomDistance: dist_max_err for prior (10, 0.009, -0.004, 50, 9

In [76]:
np.flip(np.array([[2, 3],[4,5]]))

[autoreload of dataset failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 781, in get_code
  File "<frozen importlib._bootstrap_external>", line 741, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/kfir/workspace/RadarLocalization/dataset.py", line 97
    prior_list.append({"c":

array([[5, 4],
       [3, 2]])

In [77]:
np.flip(np.array([[2, 3],[4,5]]))

array([[5, 4],
       [3, 2]])

In [20]:
import scipy.optimize

In [50]:
import numpy as np
import time

px,py = 10,23
a0, a1, a2 = 5, 0.1, 0.01
t1 = time.time()
for i in range(400):
    res = scipy.optimize.minimize(lambda x : np.sqrt((px-x)**2+(py-(a0+i/10.)+a1*x+a2*x**2)**2), x0=[5])
t2 = time.time()
print(t2-t1)
print("res",res.x)
print(res.x, a0 + a1*res.x + a2*res.x**2)
optx = res.x
opty = a0 + a1*res.x + a2*res.x**2

0.43612146377563477
res [17.71929741]
[17.71929741] [9.91166475]


In [55]:
import math
t3 = time.time()
x = np.linspace(-100,100, 100)
y = a0 + a1*x+a2*x**2
for i in range(400):
    a = np.argmin(np.sqrt((x-px)**2+(y-py)**2))
t4 = time.time()
print(t4-t3)
print(x[a], y[a])
print(math.sqrt((x[a]-px)**2+(y[a]-py)**2))
print(math.sqrt((optx-px)**2+(opty-py)**2))

0.008011102676391602
15.151515151515156 8.810835629017449
15.095379886013166
15.195133173954204


In [81]:
import numpy.polynomial.polynomial as P

In [97]:
f1 = (1, 1, 2)
f2 = (1.1, 1, 2.2)
f1mf2 = P.polysub(f1,f2)
f1mf2 = P.polymul(f1mf2,f1mf2)
val = P.polyint(f1mf2,lbnd=-10)
a = P.polyval(-10, val) + P.polyval(5,10)
print(a)

10.0


In [68]:
print (res)

[[{'f': poly1d([-1.02236445e-02,  5.34088473e-03,  3.98952978e+01]), 'x_start': -0.5529887922981365, 'x_end': 39.97303174577813, 'fxFlag': False}, {'f': poly1d([-0.00042631,  0.01510269, -0.11574659]), 'x_start': 4.833046593282244, 'x_end': 38.7783703938952, 'fxFlag': True}, {'f': poly1d([-4.46306256e-03,  6.61758397e-02,  8.85350905e+00]), 'x_start': 50.314965236442816, 'x_end': 77.68272844134319, 'fxFlag': True}], [{'f': poly1d([-1.03754735e-02,  1.29286493e-02,  3.98142964e+01]), 'x_start': -0.3503586896077549, 'x_end': 39.77040164308775, 'fxFlag': False}, {'f': poly1d([-0.0002941 ,  0.01255984, -0.13503909]), 'x_start': 4.162537176968926, 'x_end': 38.60097408217034, 'fxFlag': True}, {'f': poly1d([-1.94101401e-03, -2.51104960e-01,  1.84207231e+01]), 'x_start': 50.526461484425944, 'x_end': 85.7248176360228, 'fxFlag': True}], [{'f': poly1d([-1.03754735e-02,  1.29286493e-02,  3.98142964e+01]), 'x_start': -0.1497548879442772, 'x_end': 39.569797841424275, 'fxFlag': False}, {'f': poly1d([

In [28]:
res[1][0]["f"].c[0]

-0.010265869210950747